
# HiRAG Fiscalitate Chatbot (versiune profesională)

Acest notebook Colab pregătește un flux complet pentru a încărca textul fiscal `codfiscal.txt` și a construi un asistent RAG bazat pe **HiRAG** (modul ierarhic complet). Vei putea indexa corpusul fiscal și rula întrebări în limba română folosind același pipeline profesional din scripturile HiRAG.

Implicit, notebook-ul folosește modelul `gpt-5-mini` și embedding `text-embedding-3-large` (3072 dim), iar grafurile sunt stocate în Neo4j Aura cu conexiunile de mai jos. Poți schimba rapid furnizorul sau baza de date direct din celule.


## 1. Instalare și configurare proiectRulând celula de mai jos vei clona automat repository-ul (dacă nu există deja), vei intra în directorul lui și vei instala dependințele prin `pip install -e .` pentru a folosi implementarea oficială HiRAG.

In [ ]:
import osimport sysimport subprocessfrom pathlib import PathREPO_URL = "https://github.com/Livius2024/HiRAG.git"PROJECT_ROOT = Path.cwd()if PROJECT_ROOT.name != "HiRAG":    project_dir = PROJECT_ROOT / "HiRAG"    if not project_dir.exists():        subprocess.run(["git", "clone", REPO_URL], check=True)    os.chdir(project_dir)print(f"Using project root: {Path.cwd()}")subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-U", "pip"], check=True)subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-e", "."], check=True)


## 2. Setarea cheilor și a modelului LLM
Completează cheile API pentru furnizorul ales (OpenAI/DeepSeek/GLM). Valorile pot fi furnizate prin environment variables în Colab (`%env OPENAI_API_KEY=...`) sau direct în variabilele de mai jos. Modelul de completare și cel de embedding sunt preconfigurate pentru `gpt-5-mini` și `text-embedding-3-large` (3072 dimensiuni).


In [ ]:

import os

# Alege furnizorul LLM: 'openai', 'deepseek' sau 'glm'
PROVIDER = os.getenv("HIRAG_PROVIDER", "openai").lower()

# OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")

# DeepSeek
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY", "")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL", "https://api.deepseek.com")
DEEPSEEK_MODEL = os.getenv("DEEPSEEK_MODEL", "deepseek-chat")
DEEPSEEK_EMBEDDING_MODEL = os.getenv("DEEPSEEK_EMBEDDING_MODEL", "deepseek-embedding")

# GLM (Zhipu)
GLM_API_KEY = os.getenv("GLM_API_KEY", "")
GLM_BASE_URL = os.getenv("GLM_BASE_URL", "https://open.bigmodel.cn/api/paas/v4")
GLM_MODEL = os.getenv("GLM_MODEL", "glm-4-plus")
GLM_EMBEDDING_MODEL = os.getenv("GLM_EMBEDDING_MODEL", "embedding-3")

def ensure_key_set(provider: str):
    if provider == "openai" and not OPENAI_API_KEY:
        raise ValueError("Setează OPENAI_API_KEY înainte de a continua.")
    if provider == "deepseek" and not DEEPSEEK_API_KEY:
        raise ValueError("Setează DEEPSEEK_API_KEY înainte de a continua.")
    if provider == "glm" and not GLM_API_KEY:
        raise ValueError("Setează GLM_API_KEY înainte de a continua.")

ensure_key_set(PROVIDER)
print(f"Furnizor selectat: {PROVIDER}")



## 3. Configurarea Neo4j Aura
Completează datele de conectare pentru instanța Neo4j Aura. Valorile implicite corespund instanței `Codfiscal` furnizate, dar poți înlocui rapid cu propriile credențiale.


In [ ]:

import os

NEO4J_URI = os.getenv("NEO4J_URI", "neo4j+s://052d304f.databases.neo4j.io")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "Q0KWeSnDlidBtVCD634E532eBbBpL9Afaq-7grxCN1Y")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE", "neo4j")
AURA_INSTANCEID = os.getenv("AURA_INSTANCEID", "052d304f")
AURA_INSTANCENAME = os.getenv("AURA_INSTANCENAME", "Codfiscal")

if not NEO4J_PASSWORD:
    raise ValueError("Setează NEO4J_PASSWORD înainte de a continua.")

print(
    f"Neo4j Aura configurat pentru {NEO4J_URI} (db={NEO4J_DATABASE}, instanță={AURA_INSTANCENAME})."
)



## 4. Descărcarea corpusului fiscal
`codfiscal.txt` este luat direct din repository și salvat local. După prima rulare fișierul este reutilizat, astfel încât pașii de indexare să fie rapizi.


In [ ]:

import requests
from pathlib import Path

DATA_URL = "https://raw.githubusercontent.com/Livius2024/HiRAG/main/codfiscal.txt"
DATA_PATH = Path("data/codfiscal.txt")
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

if not DATA_PATH.exists():
    resp = requests.get(DATA_URL, timeout=60)
    resp.raise_for_status()
    DATA_PATH.write_text(resp.text, encoding="utf-8")
    print(f"Descărcat {DATA_URL} -> {DATA_PATH}")
else:
    print(f"Folosește fișierul existent: {DATA_PATH}")



## 5. Definirea funcțiilor LLM/embedding (stil HiRAG profesional)
Funcțiile de mai jos reproduc stilul din scripturile oficiale (`hi_Search_*.py`), cu caching opțional și batch-uri asincrone. Poți adapta furnizorul fără a modifica logica restului notebook-ului.


In [ ]:

import numpy as np
from dataclasses import dataclass
from openai import AsyncOpenAI
from hirag import HiRAG, QueryParam
from hirag.base import BaseKVStorage
from hirag._utils import EmbeddingFunc, compute_args_hash
from hirag._storage import NetworkXStorage, Neo4jStorage

@dataclass
class _EmbeddingWrapper(EmbeddingFunc):
    embedding_dim: int
    max_token_size: int
    func: callable

    async def __call__(self, *args, **kwargs) -> np.ndarray:
        return await self.func(*args, **kwargs)

def embedding_wrapper(**kwargs):
    def deco(func):
        return _EmbeddingWrapper(func=func, **kwargs)
    return deco

if PROVIDER == "openai":
    LLM_API_KEY = OPENAI_API_KEY
    LLM_BASE_URL = OPENAI_BASE_URL
    LLM_MODEL = OPENAI_MODEL
    EMBEDDING_MODEL = OPENAI_EMBEDDING_MODEL
    EMBEDDING_DIM = 3072 if "text-embedding-3-large" in OPENAI_EMBEDDING_MODEL else 1536
elif PROVIDER == "deepseek":
    LLM_API_KEY = DEEPSEEK_API_KEY
    LLM_BASE_URL = DEEPSEEK_BASE_URL
    LLM_MODEL = DEEPSEEK_MODEL
    EMBEDDING_MODEL = DEEPSEEK_EMBEDDING_MODEL
    EMBEDDING_DIM = 1536
else:
    LLM_API_KEY = GLM_API_KEY
    LLM_BASE_URL = GLM_BASE_URL
    LLM_MODEL = GLM_MODEL
    EMBEDDING_MODEL = GLM_EMBEDDING_MODEL
    EMBEDDING_DIM = 2048

client_async = AsyncOpenAI(api_key=LLM_API_KEY, base_url=LLM_BASE_URL)

@embedding_wrapper(embedding_dim=EMBEDDING_DIM, max_token_size=8192)
async def embedding_func(texts: list[str]) -> np.ndarray:
    result = await client_async.embeddings.create(model=EMBEDDING_MODEL, input=texts, encoding_format="float")
    return np.array([item.embedding for item in result.data])

async def chat_completion(prompt: str, system_prompt: str | None = None, history_messages=None, **kwargs) -> str:
    history_messages = history_messages or []
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.extend(history_messages)
    messages.append({"role": "user", "content": prompt})

    hashing_kv: BaseKVStorage | None = kwargs.pop("hashing_kv", None)
    args_hash = None
    if hashing_kv is not None:
        args_hash = compute_args_hash(LLM_MODEL, messages)
        cached = await hashing_kv.get_by_id(args_hash)
        if cached is not None:
            return cached["return"]

    response = await client_async.chat.completions.create(model=LLM_MODEL, messages=messages, **kwargs)
    content = response.choices[0].message.content

    if hashing_kv is not None:
        await hashing_kv.upsert({args_hash: {"return": content, "model": LLM_MODEL}})

    return content



## 6. Inițializarea grafului ierarhic HiRAG
Instanța de mai jos activează modul ierarhic (profesional), caching, embedding asincron și stocare grafică în Neo4j Aura. Poți schimba directorul de lucru pentru a păstra indecșii generați între rulări.


In [ ]:

from pathlib import Path
WORK_DIR = Path("hirag_codfiscal_cache")
WORK_DIR.mkdir(exist_ok=True)

hirag = HiRAG(
    working_dir=str(WORK_DIR),
    enable_llm_cache=True,
    enable_hierachical_mode=True,
    enable_naive_rag=True,
    embedding_func=embedding_func,
    embedding_batch_num=6,
    embedding_func_max_async=8,
    best_model_func=chat_completion,
    cheap_model_func=chat_completion,
    graph_storage_cls=Neo4jStorage,
    addon_params={
        "neo4j_url": NEO4J_URI,
        "neo4j_auth": [NEO4J_USERNAME, NEO4J_PASSWORD],
    },
)

print("HiRAG inițializat în modul ierarhic complet (Neo4j).")



## 7. Indexarea corpusului fiscal
Această celulă încarcă textul fiscal și îl inserează în graful ierarhic. Dacă rulezi de mai multe ori, indecșii existenți sunt reutilizați, iar fișierele duplicate sunt sărite automat.


In [ ]:
CODFISCAL_TEXT = DATA_PATH.read_text(encoding="utf-8")# Indexare (operație asincronă gestionată intern)hirag.insert(CODFISCAL_TEXT)print("Indexare finalizată.")


## 8. Funcție de chat și exemple de întrebări
Funcția `ask()` rulează căutarea ierarhică (`mode="hi"`) pentru răspunsuri argumentate. Poți schimba modul în `hi_bridge`, `hi_local`, `hi_global`, `hi_nobridge` sau `naive` pentru comparație.


In [ ]:
from typing import Literaldef ask(question: str, mode: Literal["hi", "hi_bridge", "hi_local", "hi_global", "hi_nobridge", "naive"] = "hi"):    print(f"Întrebare: {question}Mod: {mode}")    answer = hirag.query(question, param=QueryParam(mode=mode))    print(f"Răspuns:{answer}")# Exemplu de test: ajustează întrebarea după nevoieask("Care sunt regulile de deducere pentru TVA la achizițiile interne?", mode="hi")

> **Sugestie:** Pentru sesiuni de chat iterative, poți salva istoricul întrebărilor și să preîncarci `history_messages` în `chat_completion` pentru a menține contextul conversației.

## 9. Verificare rapidă a mediului
Poți rula o verificare simplă pentru a confirma că pachetul HiRAG se importă corect înainte de a consuma credite de inferență. Comanda de mai jos compilează toate modulele locale și semnalează eventualele erori de sintaxă.


In [ ]:
# Smoke test: validează că modulele HiRAG se compilează corect
!python -m compileall hirag
